In [ ]:
text1 = "Apples are red"
text2 = "Apples are sweet"
text3 = "Oranges are sweet"

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

sw_nltk = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

text1 = " ".join([lemmatizer.lemmatize(word) for word in nltk.word_tokenize(text1.lower()) if word not in sw_nltk])
text2 = " ".join([lemmatizer.lemmatize(word) for word in nltk.word_tokenize(text2.lower()) if word not in sw_nltk])
text3 = " ".join([lemmatizer.lemmatize(word) for word in nltk.word_tokenize(text3.lower()) if word not in sw_nltk])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tanmayshubhgarg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tanmayshubhgarg/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tanmayshubhgarg/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
total = set(text1.lower().split(" ") + text2.lower().split(" ") + text3.lower().split(" "))
print(total)
total = list(total)

{'orange', 'sweet', 'apple', 'red'}


In [ ]:
import pandas as pd

df = pd.DataFrame(0, index=total, columns=total)

In [ ]:
def makecm(corpus):
    df = pd.DataFrame(0, index=total, columns=total)
    for text in corpus:
        words = text.split(" ")
        for word in words:
            try:
                if words[1 + words.index(word)] in total:
                    df.at[words[1 + words.index(word)], word] += 1
                if words[-1 + words.index(word)] in total:
                    df.at[word, words[-1 + words.index(word)]] += 1
            except IndexError:
                continue
    return df

text1words = text1.split(" ")
for word in text1words:
    try:
        if text1words[1+text1words.index(word)] in total:
            df.at[text1words[1+text1words.index(word)], word] += 1
        if text1words[-1 + text1words.index(word)] in total:
            df.at[word, text1words[-1 + text1words.index(word)]] += 1
    except IndexError:
        continue



In [ ]:
df

,orange,sweet,apple,red
orange,0,0,0,0
sweet,0,0,0,0
apple,0,0,0,1
red,0,0,1,0


In [ ]:
text2words = text2.split(" ")
for word in text2words:
    try:
        if text2words[1+text2words.index(word)] in total:
            df.at[text2words[1+text2words.index(word)], word] += 1
        if text2words[-1 + text2words.index(word)] in total:
            df.at[word, text2words[-1 + text2words.index(word)]] += 1
    except IndexError:
        continue


In [ ]:
df

,orange,sweet,apple,red
orange,0,0,0,0
sweet,0,0,1,0
apple,0,1,0,1
red,0,0,1,0


In [ ]:
text3words = text3.split(" ")
for word in text3words:
    try:
        if text3words[1+text3words.index(word)] in total:
            df.at[text3words[1+text3words.index(word)], word] += 1
        if text3words[-1 + text3words.index(word)] in total:
            df.at[word, text3words[-1 + text3words.index(word)]] += 1
    except IndexError:
        continue

In [ ]:
df

,orange,sweet,apple,red
orange,0,1,0,0
sweet,1,0,1,0
apple,0,1,0,1
red,0,0,1,0


In [ ]:
cm = np.array(df)

In [ ]:
cm

array([[0, 1, 0, 0],
       [1, 0, 1, 0],
       [0, 1, 0, 1],
       [0, 0, 1, 0]])

In [ ]:
corpus = [text1, text2, text3]
cm = makecm(corpus)
print(cm)

        orange  sweet  apple  red
orange       0      1      0    0
sweet        1      0      1    0
apple        0      1      0    1
red          0      0      1    0


In [ ]:
cm_array = np.array(cm)

In [ ]:
cm_array

array([[0, 1, 0, 0],
       [1, 0, 1, 0],
       [0, 1, 0, 1],
       [0, 0, 1, 0]])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

array([[1, 0, 1, 0],
       [1, 0, 0, 1],
       [0, 1, 0, 1]])

In [ ]:
X = cm_array
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
dense_vector = pca.fit_transform(X)

In [ ]:
dense_vector

array([[ 0.60150096, -0.35355339],
       [-0.97324899,  0.35355339],
       [ 0.97324899,  0.35355339],
       [-0.60150096, -0.35355339]])

In [ ]:
from numpy.linalg import svd
from itertools import combinations

In [ ]:
U, s, Vt = svd(dense_vector, full_matrices=False)
U2 = U[:, :2]
S2 = np.diag(s[:2])
embeddings = U2 @ S2

In [ ]:
U2

array([[-0.37174803,  0.5       ],
       [ 0.60150096, -0.5       ],
       [-0.60150096, -0.5       ],
       [ 0.37174803,  0.5       ]])

In [ ]:
s

array([1.61803399, 0.70710678])

In [ ]:
S2

array([[1.61803399, 0.        ],
       [0.        , 0.70710678]])

In [ ]:
Vt

array([[-1., -0.],
       [-0., -1.]])

In [ ]:
embeddings

array([[-0.60150096,  0.35355339],
       [ 0.97324899, -0.35355339],
       [-0.97324899, -0.35355339],
       [ 0.60150096,  0.35355339]])

In [ ]:
def cos_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


In [ ]:
for i, j in combinations(range(len(total)), 2):
    sim = cos_sim(dense_vector[i], dense_vector[j])
    print(f"{total[i]} and {total[j]}: {sim:.4f}")

orange and sweet: -0.9833
orange and apple: 0.6373
orange and red: -0.4864
sweet and apple: -0.7668
sweet and red: 0.6373
apple and red: -0.9833


In [ ]:
for i, j in combinations(range(len(total)), 2):
    sim = cos_sim(embeddings[i], embeddings[j])
    print(f"{total[i]} and {total[j]}: {sim:.4f}")

orange and sweet: -0.9833
orange and apple: 0.6373
orange and red: -0.4864
sweet and apple: -0.7668
sweet and red: 0.6373
apple and red: -0.9833
